# Propriedades e Eventos em C++

In [1]:
#include <exception>

class Forno {
public:
    int temperatura() const {
        return _temperatura;
    }
    void temperatura(const int &t) {
        if (!(t >= 50 && t <= 250))
            throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");
        _temperatura = t;
    }
private:
    int _temperatura;
};

In [2]:
Forno forno1;

In [3]:
forno1.temperatura(50);

In [4]:
forno1.temperatura(30);

Error: 

In [5]:
std::cout << forno1.temperatura();

50

## Propriedades

Propriedades podem ser implementadas nativamente em C++ através de classes anônimas e sobrecarga de operadores.

In [6]:
class FornoComPropriedades
{
    public:
    class {
        public:
            operator int () const { return value; }

            int & operator = (const int &t) { 
                if (!(t >= 50 && t <= 250))
                    throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");
                return value = t; 
            }
        private:
            int value;

    } Temperatura;
};

In [7]:
FornoComPropriedades forno2;
forno2.Temperatura = 50;

In [8]:
forno2.Temperatura = 30;

Error: 

In [9]:
std::cout << forno2.Temperatura;

50

### Definindo propriedades genéricas por meio de macros.

In [14]:
#define ATTRIBUTE(type) \
class { \
public: \
    operator type () const { return value; } \
    type & operator = (const type &t) { return value = t;} \
private: \
    type value; \
}

In [15]:
class FornoComPropriedadesMacro
{
    public:
    ATTRIBUTE(int) Temperatura;
};

In [16]:
FornoComPropriedadesMacro forno3;
forno3.Temperatura = 50;
std::cout << forno3.Temperatura;

50

### Definindo propriedades personalizadas com ajuda de friend

Para que os métodos da classe/estrutura que contiver as propriedades possam acessar os membros privados da propriedade é necessário que as classes anônimas especifiquem um relacionamento **friend** com a classe container.

In [10]:
class FornoComPropriedadesAmigas
{
    public:
    class {
        friend class FornoComPropriedadesAmigas;
        public:
            operator int () const { return value; }

            int & operator = (const int &t) { 
                if (!(t >= 50 && t <= 250))
                    throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");
                return value = t; 
            }
        private:
            int value;

    } Temperatura;

    float fahrenheit() const {
        return Temperatura.value * 1.8f + 32.0f;
    }
    
};

In [11]:
FornoComPropriedadesAmigas forno4;
forno4.Temperatura = 50;

In [12]:
std::cout << forno4.fahrenheit();

122

### Definindo propriedades genéricas e personalizadas por meio de template

Percebemos claramente uma maior facilidade e clareza ao se programar utilizando classes construidas com propriedades, porém é necessário mais do que apenas métodos **get** e **set**. Que poderiam ser encapsulados numa classe de propriedades genérica. Para isso, vamos nos utilizar de herança e um recurso conhecido como **template**.

In [2]:
template <typename T>
class Property {
public:
    virtual ~Property() {}
    virtual T & operator = (const T &f) { return value = f; }
    virtual operator T const & () const { return value; }
protected:
    T value;
};

Agora podemos definir uma propriedade de qualquer tipo.

In [3]:
Property<int> t;
t = 50;
std::cout << t;

50

E para implementar métodos **get/set** personalizados basta usar herança.

In [4]:
class : public Property<int> {
    public:
    virtual int & operator = (const int &t) { 
        if (!(t >= 50 && t <= 250))
            throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");

        return value = t; 
    }
} t2;

In [5]:
t2 = 50;
t2 = 30;

Error: 

In [6]:
std::cout << t2;

50

In [7]:
class FornoComPropriedadesAmigasModelo {
public:
    class : public Property<int> { 
        friend class FornoComPropriedadesAmigasModelo; 
        public:
        virtual int & operator = (const int &t) { 
            if (!(t >= 50 && t <= 250))
                throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");

            return value = t; 
        }

    } Temperatura;
    
    float fahrenheit() const {
        return Temperatura.value * 1.8f + 32.0f;
    }
    
    FornoComPropriedadesAmigasModelo() { Temperatura.value = 50; }
};

In [8]:
FornoComPropriedadesAmigasModelo forno5;
forno5.Temperatura = 100;

In [9]:
std::cout << forno5.Temperatura;

100

## Eventos

Vamos ver um exemplo de eventos no funcionamento de propriedades somente leitura. Ou seja, aquelas que não permitem modificar seus valores.

In [11]:
class EventInterface
{
public:
    virtual void OnEvent(int) = 0;
};

In [10]:
template <typename T>
class ReadOnlyProperty {
public:
    virtual ~ReadOnlyProperty() {}
    virtual operator T const & () const { return value; }
protected:
    T value;
};

In [12]:
class FornoComPropriedadesAmigasModeloSomenteLeitura {
public:
    class : public Property<int> { 
        friend class FornoComPropriedadesAmigasModeloSomenteLeitura; 
        private:
            EventInterface *aoAtualizarTemperatura;
        public:
        virtual int & operator = (const int &t) { 
            if (!(t >= 50 && t <= 250))
                throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");
            if (aoAtualizarTemperatura)
                aoAtualizarTemperatura->OnEvent(t);
            return value = t; 
        }

    } Temperatura;

    class : public ReadOnlyProperty<float>, EventInterface { 
        friend class FornoComPropriedadesAmigasModeloSomenteLeitura; 
        public:
        virtual void OnEvent(int celsius) {
            value = celsius * 1.8f + 32.0f;
        }
    } Fahrenheit;

    FornoComPropriedadesAmigasModeloSomenteLeitura() { 
        Temperatura.aoAtualizarTemperatura = &Fahrenheit;
        Temperatura = 50; 
    }
};

In [13]:
FornoComPropriedadesAmigasModeloSomenteLeitura forno6;
std::cout << forno6.Fahrenheit;

122

### Definindo múltiplos eventos

In [21]:
#include <vector>

class FornoComPropriedadesMultiplas : public EventInterface {
public:
    class : public Property<int> { 
        friend class FornoComPropriedadesMultiplas; 
        private:
            std::vector<EventInterface *> listaEventos;
        public:
        virtual int & operator = (const int &t) { 
            if (!(t >= 50 && t <= 250))
                throw std::out_of_range("Temperatura deve estar entre 50 e 250 graus Celsius.");
            for (auto aoAtualizarTemperatura : listaEventos)
                if (aoAtualizarTemperatura)
                    aoAtualizarTemperatura->OnEvent(t);
            return value = t; 
        }

    } Temperatura;

    class : public ReadOnlyProperty<float>, EventInterface { 
        friend class FornoComPropriedadesMultiplas; 
        public:
        virtual void OnEvent(int celsius) {
            value = celsius * 1.8f + 32.0f;
        }
    } Fahrenheit;

    FornoComPropriedadesMultiplas() { 
        Temperatura.listaEventos.push_back(&Fahrenheit);
        Temperatura.listaEventos.push_back(this);
        Temperatura = 50; 
    }

    virtual void OnEvent(int celsius) {
        if (celsius >= 100)
            std::cout << "Temperatura acima de 100º C." ;
    }
};

In [22]:
FornoComPropriedadesMultiplas forno7;
std::cout << forno7.Fahrenheit;

122

In [23]:
forno7.Temperatura = 100;

Temperatura acima de 100º C.